In [3]:
import json

with open("./data/Training/label/EKSA900000231.json", 'r') as f:
    ann = json.load(f)

In [4]:
ann

{'sn': 'EKSA900000231',
 'origin': 'And then the third one will be related to democracy and governance.',
 'MT': '세 번째는 민주주의와 거버넌스에 관한 것입니다.',
 'MTPE': '그리고 세 번째는 민주주의와 거버넌스에 관한 것입니다.'}

In [ ]:
import os
import soundfile as sf
from datasets import Dataset
import json
from tqdm import tqdm

audio_dir = "./data/Training/wav/"
text_dir = "./data/Training/label/"

data = {"audio_path":[], "text":[]}
for file_name in tqdm(os.listdir(audio_dir)):
    if file_name.endswith(".wav"):
        audio_path = os.path.join(audio_dir, file_name)
        text_path = os.path.join(text_dir, file_name.replace(".wav", ".json"))

        with open(text_path, 'r') as f:
            transript = json.load(f)
        text = transript['MTPE']
        data['audio_path'].append(audio_path)
        data['text'].append(text)
        data['english']

100%|██████████| 68828/68828 [00:03<00:00, 21397.11it/s]


In [76]:
import librosa
import soundfile as sf

def convert_wav_librosa(input_path, output_path, target_sample_rate=16000):
    # 파일 로드 (원본 샘플링 레이트로)
    audio, sr = librosa.load(input_path, sr=None, mono=False)  # stereo 유지
    
    # 모노로 변환
    audio_mono = librosa.to_mono(audio)
    
    # 샘플링 레이트 변경
    audio_resampled = librosa.resample(audio_mono, orig_sr=sr, target_sr=target_sample_rate)
    
    # 파일 저장 (16비트 PCM으로 저장)
    sf.write(output_path, audio_resampled, target_sample_rate, subtype='PCM_16')
    print(f"파일이 변환되었습니다: {output_path}")

for p in data['audio_path']:
    convert_wav_librosa(p, p)


파일이 변환되었습니다: ./data/Training/wav/EKSA100033351.wav
파일이 변환되었습니다: ./data/Training/wav/EKND100005923.wav
파일이 변환되었습니다: ./data/Training/wav/EKND100003130.wav
파일이 변환되었습니다: ./data/Training/wav/EKND100008005.wav
파일이 변환되었습니다: ./data/Training/wav/EKSA100030374.wav
파일이 변환되었습니다: ./data/Training/wav/EKSA100058741.wav
파일이 변환되었습니다: ./data/Training/wav/EKND100008745.wav
파일이 변환되었습니다: ./data/Training/wav/EKND100002583.wav
파일이 변환되었습니다: ./data/Training/wav/EKND100012837.wav
파일이 변환되었습니다: ./data/Training/wav/EKSA100016278.wav
파일이 변환되었습니다: ./data/Training/wav/EKSA100021094.wav
파일이 변환되었습니다: ./data/Training/wav/EKSA100006793.wav
파일이 변환되었습니다: ./data/Training/wav/EKSA100027744.wav
파일이 변환되었습니다: ./data/Training/wav/EKSA100013320.wav
파일이 변환되었습니다: ./data/Training/wav/EKSA100058988.wav
파일이 변환되었습니다: ./data/Training/wav/EKSA100008029.wav
파일이 변환되었습니다: ./data/Training/wav/EKSA100058447.wav
파일이 변환되었습니다: ./data/Training/wav/EKND100013447.wav
파일이 변환되었습니다: ./data/Training/wav/EKSA100046636.wav
파일이 변환되었습니다: ./data/Training/wa

KeyboardInterrupt: 

In [25]:
import librosa
import numpy as np

def process_audio(file_path, sample_rate=16000, n_mels=80, win_length=0.025, hop_length=0.01, max_frames=3000):
    """
    WAV 파일 처리 함수: Log Mel-Filter Bank 특성 추출 및 CMVN 적용
    Args:
        file_path (str): 입력 WAV 파일 경로
        sample_rate (int): 샘플링 레이트 (Hz)
        n_mels (int): Mel 필터 뱅크 채널 수
        win_length (float): 윈도우 길이 (초 단위)
        hop_length (float): 프레임 간 이동 간격 (초 단위)
        max_frames (int): 최대 허용 프레임 수
    Returns:
        np.ndarray: CMVN이 적용된 Log Mel-Filter Bank 특성 (numpy array)
    """
    # Step 1: Load audio
    audio, sr = librosa.load(file_path, sr=sample_rate, mono=True)
    
    # Step 2: Mel-spectrogram extraction
    win_len_samples = int(win_length * sr)  # 윈도우 길이를 샘플 단위로 변환
    hop_len_samples = int(hop_length * sr)  # 홉 길이를 샘플 단위로 변환
    mel_spectrogram = librosa.feature.melspectrogram(
        y=audio, 
        sr=sr, 
        n_mels=n_mels, 
        n_fft=2048, 
        hop_length=hop_len_samples, 
        win_length=win_len_samples,
        power=2.0
    )
    
    # Log-Mel 변환
    log_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
    
    # Step 3: Utterance-level CMVN
    mean = np.mean(log_mel_spectrogram, axis=1, keepdims=True)
    std = np.std(log_mel_spectrogram, axis=1, keepdims=True)
    cmvn_features = (log_mel_spectrogram - mean) / (std + 1e-10)
    
    # Step 4: Frame filtering
    if cmvn_features.shape[1] > max_frames:
        print(f"Skipping {file_path}: Exceeds maximum frame limit ({cmvn_features.shape[1]} > {max_frames})")
        return None
    
    return cmvn_features

# 사용 예시
features = process_audio(data['audio_path'][0])
if features is not None:
    print(f"Processed features shape: {features.shape}")


Processed features shape: (80, 608)


In [30]:
features

array([[-1.4049639, -1.2569789, -1.3083081, ..., -1.4049639, -1.4049639,
        -1.4049639],
       [-1.3835788, -1.3835788, -1.3835788, ..., -1.3835788, -1.3835788,
        -1.3835788],
       [-1.5115206, -1.5115206, -1.5115206, ..., -1.5115206, -1.5115206,
        -1.5115206],
       ...,
       [-1.0690968, -1.0690968, -1.0690968, ..., -1.0690968, -1.0690968,
        -1.0690968],
       [-1.00376  , -1.00376  , -1.00376  , ..., -1.00376  , -1.00376  ,
        -1.00376  ],
       [-0.8237619, -0.8237619, -0.8237619, ..., -0.8237619, -0.8237619,
        -0.8237619]], dtype=float32)

In [69]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch

# load model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60").to('cuda')

# ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")
input_values = processor(audio_trim, return_tensors="pt", padding="longest").input_values.to('cuda')  # Batch size 1


# retrieve logits
logits = model(input_values).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60 and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [70]:
transcription

['MOST IMPORTANT HOWEVER AND THIS IS MOST INTERESTING LOOK WHAT HAPPENS HERE WHEN WE JUST PRESENT THE PEER ALONE']

In [71]:
data['text'][0]

'그러나 가장 중요한 것은 여기서 우리가 동료를 혼자 소개할 때 어떤 일이 일어나는지 보는 것입니다.'

In [74]:
input_values.shape

torch.Size([1, 94309])

In [75]:
audio_trim.shape

(94309,)

In [ ]:
audio, sr = librosa.load(data['audio_path'][0], sr=16000, mono=True)

16000

In [61]:
import numpy as np

def trim_zeros_1d(arr):
    """
    1D numpy 배열에서 뒤쪽의 0들을 제거.
    Args:
        arr (numpy.ndarray): 입력 1D 배열
    Returns:
        numpy.ndarray: 뒤쪽의 0이 제거된 배열
    """
    # 0이 아닌 마지막 인덱스 찾기
    last_nonzero_idx = np.nonzero(arr)[0][-1] if np.any(arr) else -1
    # 0 제거
    return arr[:last_nonzero_idx + 1] if last_nonzero_idx != -1 else np.array([])

audio_trim = trim_zeros_1d(audio)

In [ ]:
ds[0]["audio"]["array"]

array([-0.00048828, -0.00018311, -0.00137329, ...,  0.00079346,
        0.00091553,  0.00085449])

In [65]:
audio_trim

array([-6.1035156e-05, -3.0517578e-05, -3.0517578e-05, ...,
       -3.0517578e-05,  0.0000000e+00, -3.0517578e-05], dtype=float32)

In [27]:
from transformers import MBartForConditionalGeneration

mbart_encoder = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")

In [29]:
mbart_encoder

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [26]:
for l in mbart_encoder.encoder.layers:
    print(l.fc1)
    print(l.fc2)
    l.fc1.requries_grad=False
    l.fc2.requries_grad=False

Linear(in_features=1024, out_features=4096, bias=True)
Linear(in_features=4096, out_features=1024, bias=True)
Linear(in_features=1024, out_features=4096, bias=True)
Linear(in_features=4096, out_features=1024, bias=True)
Linear(in_features=1024, out_features=4096, bias=True)
Linear(in_features=4096, out_features=1024, bias=True)
Linear(in_features=1024, out_features=4096, bias=True)
Linear(in_features=4096, out_features=1024, bias=True)
Linear(in_features=1024, out_features=4096, bias=True)
Linear(in_features=4096, out_features=1024, bias=True)
Linear(in_features=1024, out_features=4096, bias=True)
Linear(in_features=4096, out_features=1024, bias=True)
Linear(in_features=1024, out_features=4096, bias=True)
Linear(in_features=4096, out_features=1024, bias=True)
Linear(in_features=1024, out_features=4096, bias=True)
Linear(in_features=4096, out_features=1024, bias=True)
Linear(in_features=1024, out_features=4096, bias=True)
Linear(in_features=4096, out_features=1024, bias=True)
Linear(in_

In [3]:
from transformers import Wav2Vec2Model

wav2vec= Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large")

config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

In [4]:
wav2vec

Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=1024, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Wav2Vec2Encoder(
    (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(
 

In [6]:
from sklearn.model_selection import train_test_split

x = [0,1,2,3,4,5,6,7,8,9]

t, v = train_test_split(x, shuffle=False, test_size=0.2)

In [7]:
t

[0, 1, 2, 3, 4, 5, 6, 7]

In [8]:
v

[8, 9]

In [5]:
from tqdm import tqdm
import os
import json

def prepare_dataset(split = "Training"):
    data_dir = f"./data/{split}/array/"
    text_dir = f"./data/{split}/label/"
    data = []
    for file_name in tqdm(os.listdir(data_dir)):
        path = os.path.join(data_dir, file_name)
        text_path = os.path.join(text_dir, file_name.replace(".npy", ".json"))
        with open(text_path, 'r') as f:
            t = json.load(f)
        text = t['MT']
        text_PE = t['MTPE']
        if text=="" or text_PE=="":
            print(path)
        data.append({"path": path, "translation" : text, "translation_PE" : text_PE})
    return data

data = prepare_dataset()

100%|██████████| 68815/68815 [00:03<00:00, 21639.61it/s]


In [22]:
import numpy as np

for d in data:
    p = d['path']

    speech_waveform = np.load(p)
    if speech_waveform.size == 0 or np.isnan(speech_waveform).any():
        os.remove(p)
        print(f"Deleted file: {p}")

Deleted file: ./data/Validation/array/EKSA100013640.npy
Deleted file: ./data/Validation/array/EKSA100048126.npy


In [1]:
from transformers import MBart50TokenizerFast

tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")

In [2]:
tokenizer.pad_token_id

1

In [3]:
from datasets import load_dataset
ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")


In [ ]:
for i in range(len(ds['audio'])):
    print(ds['audio'][i]['array'].shape)

(74400,)
(104560,)
(213360,)
(86720,)
(81440,)
(92960,)
(78880,)
(75760,)
(73520,)


Exception ignored from cffi callback <function SoundFile._init_virtual_io.<locals>.vio_tell at 0x7fe336e6bba0>:
Traceback (most recent call last):
  File "/data/donggunseo/anaconda3/lib/python3.12/site-packages/soundfile.py", line 1264, in vio_tell
    @_ffi.callback("sf_vio_tell")

KeyboardInterrupt: 


(137920,)
(172160,)
(80240,)
(123840,)
(26239,)
(116880,)
(90880,)
(29600,)
(59760,)
(96000,)
(75200,)
(69120,)
(74400,)
(133600,)
(138560,)


In [10]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch
import numpy as np
import json

# load model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60")

# load dummy dataset and read soundfiles
ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")

# tokenize
wave_form = np.load('./data/Training/array/EKND100000018.npy')
input_values = processor(wave_form, return_tensors="pt", padding="longest").input_values  # Batch size 1


print(input_values.shape)

# retrieve logits
logits = model(input_values).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

with open('./data/Training/label/EKND100000018.json', 'r') as f:
    data = json.load(f)
ref = data['MT']
ref_PE = data['MTPE']


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60 and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


torch.Size([1, 74112])


In [11]:
transcription

['POSITION YOUR FINGERS AND THUMB ON THE ANCHOR DISPENSER SIMILAR TO HOLDING A SYRINGE']

In [ ]:
ref

'주사기를 잡는 것과 유사하게 앵커 디스펜서에 손가락과 엄지손가락을 놓습니다.'

In [13]:
ref_PE

'주사기를 잡는 것과 유사하게 앵커 디스펜서에 손가락과 엄지손가락을 놓습니다.'

In [14]:
data['origin']

'Position your fingers and thumb on the anchor dispenser similar to holding a syringe.'

In [16]:
import torch

x = torch.randn(32,128)

In [20]:
x[:,1:][0]

tensor([-0.1119,  2.8685,  0.5346, -0.6251,  1.1159, -2.1839,  1.0379,  1.2871,
         1.7197,  0.3269,  0.3784,  0.2106,  1.5186, -1.5066, -0.1291, -0.3295,
         0.2719, -0.5605,  0.1989, -0.0471,  1.7175,  0.4771, -0.1685,  0.3337,
         2.8373,  1.0013,  0.5075,  0.8166,  0.8938, -0.4087, -1.1834, -0.6861,
        -0.8330, -1.3145, -0.3095,  0.1451,  0.1109,  0.4733, -0.7727, -0.9069,
        -1.3880, -0.7259, -0.3845,  0.8206,  0.1639, -0.6004, -0.1831, -0.2955,
         0.0940,  0.0785, -0.5782,  1.0870, -0.7413,  0.0225,  0.4617,  0.0497,
         0.6918,  2.3707,  0.4979,  0.5847,  0.0826,  0.5261,  0.2776, -0.5965,
        -0.2799,  1.0623, -0.3220,  0.4774, -0.4409,  0.2723,  0.9704,  0.2883,
        -0.7946, -0.4199, -0.7179, -0.2224, -1.2211, -0.6546,  0.6328,  0.1601,
        -0.7230,  1.1755, -0.6145,  0.1905,  1.4607,  1.1006, -0.1317,  1.4474,
        -0.4756,  0.4084,  0.5679,  0.3082,  0.3161,  0.8755, -0.4694, -0.9945,
         1.4780, -0.7992, -1.7711, -1.29

In [19]:
x[0]

tensor([-0.2636, -0.1119,  2.8685,  0.5346, -0.6251,  1.1159, -2.1839,  1.0379,
         1.2871,  1.7197,  0.3269,  0.3784,  0.2106,  1.5186, -1.5066, -0.1291,
        -0.3295,  0.2719, -0.5605,  0.1989, -0.0471,  1.7175,  0.4771, -0.1685,
         0.3337,  2.8373,  1.0013,  0.5075,  0.8166,  0.8938, -0.4087, -1.1834,
        -0.6861, -0.8330, -1.3145, -0.3095,  0.1451,  0.1109,  0.4733, -0.7727,
        -0.9069, -1.3880, -0.7259, -0.3845,  0.8206,  0.1639, -0.6004, -0.1831,
        -0.2955,  0.0940,  0.0785, -0.5782,  1.0870, -0.7413,  0.0225,  0.4617,
         0.0497,  0.6918,  2.3707,  0.4979,  0.5847,  0.0826,  0.5261,  0.2776,
        -0.5965, -0.2799,  1.0623, -0.3220,  0.4774, -0.4409,  0.2723,  0.9704,
         0.2883, -0.7946, -0.4199, -0.7179, -0.2224, -1.2211, -0.6546,  0.6328,
         0.1601, -0.7230,  1.1755, -0.6145,  0.1905,  1.4607,  1.1006, -0.1317,
         1.4474, -0.4756,  0.4084,  0.5679,  0.3082,  0.3161,  0.8755, -0.4694,
        -0.9945,  1.4780, -0.7992, -1.77